## Import necessary Libraries

In [253]:
import requests
import json
import pandas as pd
import psycopg2
import csv

### Extraction layer

In [ ]:
url = "https://realty-mole-property-api.p.rapidapi.com/randomProperties"

querystring = {"limit":"500"}

headers = {
	"X-RapidAPI-Key": "7fc521eee3msh22f7b3551ecef82p100f89jsn71a07ea1c1d6",
	"X-RapidAPI-Host": "realty-mole-property-api.p.rapidapi.com"
}

response = requests.get(url, headers=headers, params=querystring)

print(response.json())

response.json()

In [3]:
data = response.json()

filename = 'real_estate.json'

with open(filename, 'w') as file:
    json.dump(data, file, indent=4)

In [4]:
# loading the data into pandas DataFrame
real_estate_df= pd.read_json('real_estate.json')


In [ ]:
real_estate_df.info()

In [7]:
real_estate_df.columns

Index(['bathrooms', 'bedrooms', 'squareFootage', 'county', 'propertyType',
       'addressLine1', 'city', 'state', 'zipCode', 'formattedAddress',
       'yearBuilt', 'features', 'assessorID', 'legalDescription',
       'subdivision', 'zoning', 'lotSize', 'taxAssessment', 'propertyTaxes',
       'lastSalePrice', 'lastSaleDate', 'owner', 'id', 'longitude', 'latitude',
       'ownerOccupied', 'addressLine2'],
      dtype='object')

In [ ]:
real_estate_df.head()

In [9]:
location_dim= real_estate_df[['id','addressLine1','addressLine2', 'city', 'state', 'zipCode', 'formattedAddress',
                              'county','longitude', 'latitude']].copy().drop_duplicates().reset_index(drop=True)

In [187]:
features_dim= pd.json_normalize(real_estate_df['features'])
features_dim['id']=real_estate_df['id']
features_dim['bathrooms'] = real_estate_df['bathrooms']
features_dim['bedrooms'] = real_estate_df['bedrooms']
features_dim['squareFootage'] = real_estate_df['squareFootage']
features_dim= features_dim[['id','architectureType', 'exteriorType', 'floorCount', 'heating', 'viewType',
                          'cooling', 'coolingType', 'fireplace', 'garage', 'garageType',
                          'heatingType', 'roofType', 'roomCount', 'unitCount', 'fireplaceType',
                          'foundationType', 'garageSpaces', 'pool', 'poolType','bathrooms','bedrooms','squareFootage']].copy().drop_duplicates().reset_index(drop=True)
features_dim.head(2)


,id,architectureType,exteriorType,floorCount,heating,viewType,cooling,coolingType,fireplace,garage,...,roomCount,unitCount,fireplaceType,foundationType,garageSpaces,pool,poolType,bathrooms,bedrooms,squareFootage
0,"6522-28th-Ave,-Kenosha,-WI-53143",Duplex Single,Aluminum / Vinyl Siding,2.0,True,NaN,True,Commercial,NaN,True,...,NaN,2.0,NaN,NaN,NaN,NaN,NaN,2.0,6.0,1848.0
1,"20422-W-Legend-Trl,-Buckeye,-AZ-85396",NaN,Stucco,1.0,True,NaN,NaN,NaN,NaN,True,...,NaN,NaN,NaN,NaN,3.0,NaN,NaN,3.0,3.0,2566.0


In [235]:
sales_dim= real_estate_df[['id', 'lastSalePrice',
       'lastSaleDate' ]].copy().drop_duplicates().reset_index(drop=True)

In [368]:
property_fact = real_estate_df[['id','assessorID','legalDescription', 'ownerOccupied',
                                'yearBuilt', 'zoning', 'lotSize', 'propertyType','subdivision']]

In [369]:
property_fact.head(5)

,id,assessorID,legalDescription,ownerOccupied,yearBuilt,zoning,lotSize,propertyType,subdivision
0,"6522-28th-Ave,-Kenosha,-WI-53143",01-122-01-156-015,LOT 6 BLK 3 PUGH'S SUB PT OF NE 1/4 SEC 1 T 1 ...,NaN,1916.0,M-1,6534.0,Duplex-Triplex,PUGHS SUB
1,"20422-W-Legend-Trl,-Buckeye,-AZ-85396",502-83-361,VERRADO PHASE 3 SOUTH PHASES A AND B MCR 1156-40,1.0,2018.0,PC,9952.0,Single Family,VERRADO PH3
2,"1041-Scaleybark-Rd,-Charlotte,-NC-28209",14918213,DIST:1 CITY/MUNI/TWP:CHARLOTTE MAP REF:BK 149 ...,1.0,1956.0,R-4,21683.0,Single Family,DILWORTH
3,"2352-Sw-5th-St,-Apt-16,-Miami,-FL-33135",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"2301-S-Austin-Ave,-Ste-5,-Denison,-TX-75020",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [30]:
Tax_Assessment_dim.columns

Index(['2021.value', '2021.land', '2021.improvements', '2023.value',
       '2023.land', '2023.improvements', '2022.value', '2022.land',
       '2022.improvements', '2020.value', '2020.land', '2020.improvements',
       '2019.value', '2019.land', '2024.value', '2024.land',
       '2024.improvements', '2018.value', '2018.land', '2018.improvements',
       '2019.improvements', '2017.value', '2010.value', '2010.land',
       '2010.improvements', '2025.value', '2025.land', '2025.improvements'],
      dtype='object')

In [193]:
Tax_Assessment_dim= pd.json_normalize(real_estate_df['taxAssessment'])
Tax_Assessment_dim['id']= real_estate_df['id']
Tax_Assessment_dim= Tax_Assessment_dim[['id','2021.value', '2021.land', '2021.improvements', '2023.value',
                                        '2023.land', '2023.improvements', '2022.value', '2022.land',
                                        '2022.improvements', '2020.value', '2020.land', '2020.improvements',
                                        '2019.value', '2019.land', '2024.value', '2024.land',
                                        '2024.improvements', '2018.value', '2018.land', '2018.improvements',
                                        '2019.improvements', '2017.value', '2010.value', '2010.land',
                                        '2010.improvements', '2025.value', '2025.land', '2025.improvements']]
Tax_Assessment_dim.head(3)


,id,2021.value,2021.land,2021.improvements,2023.value,2023.land,2023.improvements,2022.value,2022.land,2022.improvements,...,2018.land,2018.improvements,2019.improvements,2017.value,2010.value,2010.land,2010.improvements,2025.value,2025.land,2025.improvements
0,"6522-28th-Ave,-Kenosha,-WI-53143",97700.0,17100.0,80600.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"20422-W-Legend-Trl,-Buckeye,-AZ-85396",31550.0,6310.0,25240.0,43570.0,8710.0,34860.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"1041-Scaleybark-Rd,-Charlotte,-NC-28209",NaN,NaN,NaN,474500.0,445500.0,29000.0,362600.0,216000.0,146600.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [194]:
Tax_Assessment_dim.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 29 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 500 non-null    object 
 1   2021.value         84 non-null     float64
 2   2021.land          68 non-null     float64
 3   2021.improvements  71 non-null     float64
 4   2023.value         105 non-null    float64
 5   2023.land          91 non-null     float64
 6   2023.improvements  93 non-null     float64
 7   2022.value         192 non-null    float64
 8   2022.land          153 non-null    float64
 9   2022.improvements  156 non-null    float64
 10  2020.value         75 non-null     float64
 11  2020.land          62 non-null     float64
 12  2020.improvements  67 non-null     float64
 13  2019.value         62 non-null     float64
 14  2019.land          50 non-null     float64
 15  2024.value         8 non-null      float64
 16  2024.land          4 non-n

In [195]:
propert_tax_dim =pd.json_normalize(real_estate_df['propertyTaxes'])
propert_tax_dim['id']= real_estate_df['id']
propert_tax_dim=propert_tax_dim[['id','2020.total', '2021.total', '2022.total', '2023.total', '2019.total',
                                 '2017.total', '2018.total', '2016.total']]
propert_tax_dim.head(5)

,id,2020.total,2021.total,2022.total,2023.total,2019.total,2017.total,2018.total,2016.total
0,"6522-28th-Ave,-Kenosha,-WI-53143",NaN,2198.0,NaN,NaN,NaN,NaN,NaN,NaN
1,"20422-W-Legend-Trl,-Buckeye,-AZ-85396",3133.0,NaN,3321.0,NaN,NaN,NaN,NaN,NaN
2,"1041-Scaleybark-Rd,-Charlotte,-NC-28209",NaN,NaN,3499.0,4579.0,NaN,NaN,NaN,NaN
3,"2352-Sw-5th-St,-Apt-16,-Miami,-FL-33135",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"2301-S-Austin-Ave,-Ste-5,-Denison,-TX-75020",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [196]:
owner_dim.columns

Index(['id', 'names', 'mailingAddress.id', 'mailingAddress.addressLine1',
       'mailingAddress.city', 'mailingAddress.state', 'mailingAddress.zipCode',
       'mailingAddress.formattedAddress', 'mailingAddress.addressLine2'],
      dtype='object')

In [324]:
owner_dim= pd.json_normalize(real_estate_df['owner'])
owner_dim['id'] = real_estate_df['id']
owner_dim= owner_dim[['id','names', 'mailingAddress.id','mailingAddress.addressLine1',
                    'mailingAddress.city', 'mailingAddress.state', 'mailingAddress.zipCode',
                    'mailingAddress.formattedAddress', 'mailingAddress.addressLine2']]

owner_dim.head(5)

,id,names,mailingAddress.id,mailingAddress.addressLine1,mailingAddress.city,mailingAddress.state,mailingAddress.zipCode,mailingAddress.formattedAddress,mailingAddress.addressLine2
0,"6522-28th-Ave,-Kenosha,-WI-53143",[TIMOTHY SMITH],"481-Broadview-Ave,-Highland-Park,-IL-60035",481 Broadview Ave,Highland Park,IL,60035,NaN,NaN
1,"20422-W-Legend-Trl,-Buckeye,-AZ-85396",[DARBY SPARKS],"20422-W-Legend-Trl,-Buckeye,-AZ-85396",20422 W Legend Trl,Buckeye,AZ,85396,NaN,NaN
2,"1041-Scaleybark-Rd,-Charlotte,-NC-28209",[JOSE LUIS VIVEROS VIVEROS],"1041-Scaleybark-Rd,-Charlotte,-NC-28209",1041 Scaleybark Rd,Charlotte,NC,28209,NaN,NaN
3,"2352-Sw-5th-St,-Apt-16,-Miami,-FL-33135",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"2301-S-Austin-Ave,-Ste-5,-Denison,-TX-75020",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Cleaning my Dataset

In [198]:
location_dim.info()
location_dim.fillna({
    'addressLine2':'unknown'
}, inplace=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                500 non-null    object 
 1   addressLine1      500 non-null    object 
 2   addressLine2      500 non-null    object 
 3   city              500 non-null    object 
 4   state             500 non-null    object 
 5   zipCode           500 non-null    int64  
 6   formattedAddress  500 non-null    object 
 7   county            500 non-null    object 
 8   longitude         500 non-null    float64
 9   latitude          500 non-null    float64
dtypes: float64(2), int64(1), object(7)
memory usage: 39.2+ KB


In [199]:
location_dim.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                500 non-null    object 
 1   addressLine1      500 non-null    object 
 2   addressLine2      500 non-null    object 
 3   city              500 non-null    object 
 4   state             500 non-null    object 
 5   zipCode           500 non-null    int64  
 6   formattedAddress  500 non-null    object 
 7   county            500 non-null    object 
 8   longitude         500 non-null    float64
 9   latitude          500 non-null    float64
dtypes: float64(2), int64(1), object(7)
memory usage: 39.2+ KB


In [408]:
features_dim.info()
features_dim.fillna({
   'architectureType':'unknown',
    'exteriorType' : 'unknown',
    'floorCount' : 0.0,
    'heating' : 'unknown',
    'viewType' : 'unknown',
    'cooling' : 'unknown',
    'coolingType' : 'unknown',
    'fireplace' : 'unknown',
    'garage' : 'unknown',
    'garageType' : 'unknown',
    'heatingType' : 'unknown',
    'roofType' : 'unknown',
    'roomCount' : 0.0,
    'unitCount' : 0.0,
    'fireplaceType' : 'unknown',
    'foundationType' : 'unknown',
    'garageSpaces' : 0.0,
    'pool' : 'unknown',
    'poolType' : 'unknown',
    'bedrooms' : 0.0,
    'squareFootage' : 0.0,
    'bathrooms': 0.0
}, inplace=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 23 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                500 non-null    object 
 1   architectureType  500 non-null    object 
 2   exteriorType      500 non-null    object 
 3   floorCount        500 non-null    float64
 4   heating           500 non-null    object 
 5   viewType          500 non-null    object 
 6   cooling           500 non-null    object 
 7   coolingType       500 non-null    object 
 8   fireplace         500 non-null    object 
 9   garage            500 non-null    object 
 10  garageType        500 non-null    object 
 11  heatingType       500 non-null    object 
 12  roofType          500 non-null    object 
 13  roomCount         500 non-null    float64
 14  unitCount         500 non-null    float64
 15  fireplaceType     500 non-null    object 
 16  foundationType    500 non-null    object 
 1

In [409]:
features_dim.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 23 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                500 non-null    object 
 1   architectureType  500 non-null    object 
 2   exteriorType      500 non-null    object 
 3   floorCount        500 non-null    float64
 4   heating           500 non-null    object 
 5   viewType          500 non-null    object 
 6   cooling           500 non-null    object 
 7   coolingType       500 non-null    object 
 8   fireplace         500 non-null    object 
 9   garage            500 non-null    object 
 10  garageType        500 non-null    object 
 11  heatingType       500 non-null    object 
 12  roofType          500 non-null    object 
 13  roomCount         500 non-null    float64
 14  unitCount         500 non-null    float64
 15  fireplaceType     500 non-null    object 
 16  foundationType    500 non-null    object 
 1

In [411]:
features_dim.head(5)

,id,architectureType,exteriorType,floorCount,heating,viewType,cooling,coolingType,fireplace,garage,...,roomCount,unitCount,fireplaceType,foundationType,garageSpaces,pool,poolType,bathrooms,bedrooms,squareFootage
0,"6522-28th-Ave,-Kenosha,-WI-53143",Duplex Single,Aluminum / Vinyl Siding,2.0,True,unknown,True,Commercial,unknown,True,...,0.0,2.0,unknown,unknown,0.0,unknown,unknown,2.0,6.0,1848.0
1,"20422-W-Legend-Trl,-Buckeye,-AZ-85396",unknown,Stucco,1.0,True,unknown,unknown,unknown,unknown,True,...,0.0,0.0,unknown,unknown,3.0,unknown,unknown,3.0,3.0,2566.0
2,"1041-Scaleybark-Rd,-Charlotte,-NC-28209",unknown,Brick Veneer,1.0,True,unknown,True,Central,True,True,...,0.0,1.0,Single,Raised,0.0,unknown,unknown,2.0,3.0,2484.0
3,"2352-Sw-5th-St,-Apt-16,-Miami,-FL-33135",unknown,unknown,0.0,unknown,unknown,unknown,unknown,unknown,unknown,...,0.0,0.0,unknown,unknown,0.0,unknown,unknown,0.0,0.0,0.0
4,"2301-S-Austin-Ave,-Ste-5,-Denison,-TX-75020",unknown,unknown,0.0,unknown,unknown,unknown,unknown,unknown,unknown,...,0.0,0.0,unknown,unknown,0.0,unknown,unknown,0.0,0.0,0.0


In [ ]:
property_fact.info()

In [372]:
property_fact.fillna({
   'assessorID' : 'unknown',
   'legalDescription' : 'unknown',
   'ownerOccupied' : 0.0,
   'yearBuilt' : 0,
   'zoning' : 'unknown',
   'lotSize' : 0.0,
   'propertyType' : 'unknown',
   'subdivision' : 'unknown'
}, inplace=True)

C:\Users\HP\AppData\Local\Temp\ipykernel_15508\2979261574.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  property_fact.fillna({


In [373]:
property_fact['yearBuilt'] = property_fact['yearBuilt'].astype(int)

C:\Users\HP\AppData\Local\Temp\ipykernel_15508\1367370935.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  property_fact['yearBuilt'] = property_fact['yearBuilt'].astype(int)


In [374]:
property_fact.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                500 non-null    object 
 1   assessorID        500 non-null    object 
 2   legalDescription  500 non-null    object 
 3   ownerOccupied     500 non-null    float64
 4   yearBuilt         500 non-null    int32  
 5   zoning            500 non-null    object 
 6   lotSize           500 non-null    float64
 7   propertyType      500 non-null    object 
 8   subdivision       500 non-null    object 
dtypes: float64(2), int32(1), object(6)
memory usage: 33.3+ KB


In [375]:
property_fact.head(10)

,id,assessorID,legalDescription,ownerOccupied,yearBuilt,zoning,lotSize,propertyType,subdivision
0,"6522-28th-Ave,-Kenosha,-WI-53143",01-122-01-156-015,LOT 6 BLK 3 PUGH'S SUB PT OF NE 1/4 SEC 1 T 1 ...,0.0,1916,M-1,6534.0,Duplex-Triplex,PUGHS SUB
1,"20422-W-Legend-Trl,-Buckeye,-AZ-85396",502-83-361,VERRADO PHASE 3 SOUTH PHASES A AND B MCR 1156-40,1.0,2018,PC,9952.0,Single Family,VERRADO PH3
2,"1041-Scaleybark-Rd,-Charlotte,-NC-28209",14918213,DIST:1 CITY/MUNI/TWP:CHARLOTTE MAP REF:BK 149 ...,1.0,1956,R-4,21683.0,Single Family,DILWORTH
3,"2352-Sw-5th-St,-Apt-16,-Miami,-FL-33135",unknown,unknown,0.0,0,unknown,0.0,unknown,unknown
4,"2301-S-Austin-Ave,-Ste-5,-Denison,-TX-75020",unknown,unknown,0.0,0,unknown,0.0,unknown,unknown
5,"51-Birdsong-Pkwy,-Orchard-Park,-NY-14127",146089 173.12-1-22,LOT:22.000 BLK:1 SEC:173.120 DIST:146089 CITY/...,1.0,2005,R1,25700.0,Single Family,HOLLAND LAND COMPANYS SURVEY
6,"113-Oak-Ave,-Bellmawr,-NJ-08031",unknown,unknown,0.0,0,unknown,0.0,Single Family,unknown
7,"11875-Westview-Pkwy,-San-Diego,-CA-92126",318-570-21-05,PM18533 PAR 1*US 24PER DOC01-26283&UND INT IN,1.0,2001,R-1:SINGLE FAM-RES,82553.0,Condo,TAMPICO
8,"8715-Shadow-Crest-St,-Houston,-TX-77074",0872510000003,LT 3 BLK 15 BRAEBURN GLEN SEC 1,1.0,1958,unknown,7148.0,Single Family,BRAEBURN GLEN
9,"4465-Boca-Way,-Spc-137,-Reno,-NV-89502",021-624-39,DONNER SPRINGS MH LT 11 BLK D,1.0,1979,MF14,5271.0,Manufactured,DONNER SPRINGS MOBILE HOMES


In [204]:
Tax_Assessment_dim.info()
Tax_Assessment_dim.fillna({

    '2022.value' : 0.0, 
    '2022.land' : 0.0, 
    '2022.improvements' : 0.0,
    '2021.value' : 0.0,
    '2021.land' : 0.0, 
    '2021.improvements' : 0.0,
    '2020.value' : 0.0, 
    '2020.land' : 0.0,
    '2020.improvements' : 0.0, 
    '2023.value' : 0.0,
    '2024.value' : 0.0, 
    '2024.improvements': 0.0,
    '2024.land' : 0.0,
    '2023.land' : 0.0,
    '2023.improvements' : 0.0, 
    '2019.value' : 0.0, 
    '2019.land' : 0.0, 
    '2019.improvements' : 0.0,
    '2017.value' : 0.0, 
    '2017.land' : 0.0, 
    '2017.improvements' : 0.0,
    '2018.value' : 0.0,
    '2018.land' : 0.0, 
    '2025.land' : 0.0,
    '2025.improvements' : 0.0,
    '2018.improvements' : 0.0,
    '2010.value' : 0.0,
    '2010.land' : 0.0,
    '2010.improvements' : 0.0,
    '2025.value' : 0.0
}, inplace=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 29 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 500 non-null    object 
 1   2021.value         84 non-null     float64
 2   2021.land          68 non-null     float64
 3   2021.improvements  71 non-null     float64
 4   2023.value         105 non-null    float64
 5   2023.land          91 non-null     float64
 6   2023.improvements  93 non-null     float64
 7   2022.value         192 non-null    float64
 8   2022.land          153 non-null    float64
 9   2022.improvements  156 non-null    float64
 10  2020.value         75 non-null     float64
 11  2020.land          62 non-null     float64
 12  2020.improvements  67 non-null     float64
 13  2019.value         62 non-null     float64
 14  2019.land          50 non-null     float64
 15  2024.value         8 non-null      float64
 16  2024.land          4 non-n

In [205]:
Tax_Assessment_dim.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 29 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 500 non-null    object 
 1   2021.value         500 non-null    float64
 2   2021.land          500 non-null    float64
 3   2021.improvements  500 non-null    float64
 4   2023.value         500 non-null    float64
 5   2023.land          500 non-null    float64
 6   2023.improvements  500 non-null    float64
 7   2022.value         500 non-null    float64
 8   2022.land          500 non-null    float64
 9   2022.improvements  500 non-null    float64
 10  2020.value         500 non-null    float64
 11  2020.land          500 non-null    float64
 12  2020.improvements  500 non-null    float64
 13  2019.value         500 non-null    float64
 14  2019.land          500 non-null    float64
 15  2024.value         500 non-null    float64
 16  2024.land          500 non

In [341]:
Tax_Assessment_dim.columns

Index(['id', '2021.value', '2021.land', '2021.improvements', '2023.value',
       '2023.land', '2023.improvements', '2022.value', '2022.land',
       '2022.improvements', '2020.value', '2020.land', '2020.improvements',
       '2019.value', '2019.land', '2024.value', '2024.land',
       '2024.improvements', '2018.value', '2018.land', '2018.improvements',
       '2019.improvements', '2017.value', '2010.value', '2010.land',
       '2010.improvements', '2025.value', '2025.land', '2025.improvements'],
      dtype='object')

In [342]:
Tax_Assessment_dim.rename(columns={'2021.value': '2021_value', 
                                   '2021.land':'2021_land', 
                                   '2021.improvements':'2021_improvements', 
                                   '2023.value':'2023_value',
                                    '2023.land':'2023_land', 
                                    '2023.improvements':'2023_improvements', 
                                    '2022.value':'2022_value', 
                                    '2022.land':'2022_land',
                                    '2022.improvements':'2022_improvements',
                                    '2020.value':'2020_value', 
                                    '2020.land':'2020_land', 
                                    '2020.improvements':'2020_improvements',
                                    '2019.value':'2019_value', 
                                    '2019.land':'2019_land', 
                                    '2024.value':'2024_value', 
                                    '2024.land':'2024_land',
                                    '2024.improvements':'2024_improvements', 
                                    '2018.value':'2018_value', 
                                    '2018.land':'2018_land', 
                                    '2018.improvements':'2018_improvements',
                                    '2019.improvements':'2019_improvements',
                                    '2017.value':'2017_value', 
                                    '2010.value':'2010_value', 
                                    '2010.land':'2010_land',
                                    '2010.improvements':'2010_improvements', 
                                    '2025.value':'2025_value', 
                                    '2025.land':'2025_land', 
                                    '2025.improvements':'2025_improvements'}, inplace=True)

In [436]:
Tax_Assessment_dim.columns

Index(['id', '2021_value', '2021_land', '2021_improvements', '2023_value',
       '2023_land', '2023_improvements', '2022_value', '2022_land',
       '2022_improvements', '2020_value', '2020_land', '2020_improvements',
       '2019_value', '2019_land', '2024_value', '2024_land',
       '2024_improvements', '2018_value', '2018_land', '2018_improvements',
       '2019_improvements', '2017_value', '2010_value', '2010_land',
       '2010_improvements', '2025_value', '2025_land', '2025_improvements'],
      dtype='object')

In [206]:
propert_tax_dim.info()
propert_tax_dim.fillna({
     
    '2020.total' : 0.0,
    '2021.total' : 0.0,
    '2022.total' : 0.0,
    '2023.total' : 0.0,
    '2019.total' : 0.0,
    '2017.total' : 0.0,
    '2018.total' : 0.0,
    '2016.total' : 0.0
}, inplace=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   id          500 non-null    object 
 1   2020.total  80 non-null     float64
 2   2021.total  79 non-null     float64
 3   2022.total  198 non-null    float64
 4   2023.total  55 non-null     float64
 5   2019.total  61 non-null     float64
 6   2017.total  1 non-null      float64
 7   2018.total  20 non-null     float64
 8   2016.total  2 non-null      float64
dtypes: float64(8), object(1)
memory usage: 35.3+ KB


In [337]:
propert_tax_dim.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   id          500 non-null    object 
 1   2020_total  500 non-null    float64
 2   2021_total  500 non-null    float64
 3   2022_total  500 non-null    float64
 4   2023_total  500 non-null    float64
 5   2019.total  500 non-null    float64
 6   2017.total  500 non-null    float64
 7   2018.total  500 non-null    float64
 8   2016.total  500 non-null    float64
dtypes: float64(8), object(1)
memory usage: 35.3+ KB


In [338]:
propert_tax_dim.rename(columns={'2020.total':'2020_total',
                                 '2021.total':'2021_total',
                                 '2022.total':'2022_total',
                                 '2023.total':'2023_total',
                                 '2019.total':'2019_total',
                                 '2017.total':'2017_total',
                                 '2018.total':'2018_total',
                                 '2016.total':'2016_total'}, inplace=True)

In [339]:
propert_tax_dim.columns

Index(['id', '2020_total', '2021_total', '2022_total', '2023_total',
       '2019_total', '2017_total', '2018_total', '2016_total'],
      dtype='object')

In [340]:
propert_tax_dim.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   id          500 non-null    object 
 1   2020_total  500 non-null    float64
 2   2021_total  500 non-null    float64
 3   2022_total  500 non-null    float64
 4   2023_total  500 non-null    float64
 5   2019_total  500 non-null    float64
 6   2017_total  500 non-null    float64
 7   2018_total  500 non-null    float64
 8   2016_total  500 non-null    float64
dtypes: float64(8), object(1)
memory usage: 35.3+ KB


In [208]:
owner_dim.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 9 columns):
 #   Column                           Non-Null Count  Dtype 
---  ------                           --------------  ----- 
 0   id                               500 non-null    object
 1   names                            325 non-null    object
 2   mailingAddress.id                319 non-null    object
 3   mailingAddress.addressLine1      319 non-null    object
 4   mailingAddress.city              319 non-null    object
 5   mailingAddress.state             319 non-null    object
 6   mailingAddress.zipCode           319 non-null    object
 7   mailingAddress.formattedAddress  50 non-null     object
 8   mailingAddress.addressLine2      25 non-null     object
dtypes: object(9)
memory usage: 35.3+ KB


In [325]:
owner_dim.info()
owner_dim.fillna({
   'names' : 'unknwon',
   'mailingAddress.id' : 'unknwon',
   'mailingAddress.addressLine1' : 'unknwon',
   'mailingAddress.city' : 'unknwon',
   'mailingAddress.state' : 'unknwon',
   'mailingAddress.zipCode' : 'unknwon',
   'mailingAddress.formattedAddress' : 'unknwon',
   'mailingAddress.addressLine2' : 'unknwon'
}, inplace=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 9 columns):
 #   Column                           Non-Null Count  Dtype 
---  ------                           --------------  ----- 
 0   id                               500 non-null    object
 1   names                            325 non-null    object
 2   mailingAddress.id                319 non-null    object
 3   mailingAddress.addressLine1      319 non-null    object
 4   mailingAddress.city              319 non-null    object
 5   mailingAddress.state             319 non-null    object
 6   mailingAddress.zipCode           319 non-null    object
 7   mailingAddress.formattedAddress  50 non-null     object
 8   mailingAddress.addressLine2      25 non-null     object
dtypes: object(9)
memory usage: 35.3+ KB


In [326]:
#removing the dot(.) in column name
owner_dim.rename(columns={'mailingAddress.id' : 'mailingAddress_id',
                          'mailingAddress.addressLine1' : 'mailingAddress_addressLine1', 
                          'mailingAddress.city' : 'mailingAddress_city',
                          'mailingAddress.state' : 'mailingAddress_state',
                          'mailingAddress.zipCode' : 'mailingAddress_zipCode',
                          'mailingAddress.formattedAddress' : 'mailingAddress_formattedAddress',
                          'mailingAddress.addressLine2' : 'mailingAddress_addressLine2'}, inplace=True)

In [327]:
owner_dim.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 9 columns):
 #   Column                           Non-Null Count  Dtype 
---  ------                           --------------  ----- 
 0   id                               500 non-null    object
 1   names                            500 non-null    object
 2   mailingAddress_id                500 non-null    object
 3   mailingAddress_addressLine1      500 non-null    object
 4   mailingAddress_city              500 non-null    object
 5   mailingAddress_state             500 non-null    object
 6   mailingAddress_zipCode           500 non-null    object
 7   mailingAddress_formattedAddress  500 non-null    object
 8   mailingAddress_addressLine2      500 non-null    object
dtypes: object(9)
memory usage: 35.3+ KB


In [223]:
sales_dim


,id,lastSalePrice,lastSaleDate
0,"6522-28th-Ave,-Kenosha,-WI-53143",194400.0,2022-03-17T00:00:00.000Z
1,"20422-W-Legend-Trl,-Buckeye,-AZ-85396",654551.0,2022-02-08T00:00:00.000Z
2,"1041-Scaleybark-Rd,-Charlotte,-NC-28209",29000.0,2018-11-28T00:00:00.000Z
3,"2352-Sw-5th-St,-Apt-16,-Miami,-FL-33135",NaN,NaN
4,"2301-S-Austin-Ave,-Ste-5,-Denison,-TX-75020",NaN,NaN
...,...,...,...
495,"4401-Cedar-Ave,-Pikesville,-MD-21208",128000.0,2016-11-14T00:00:00.000Z
496,"7945-Cobblesprings-Dr,-Avon,-IN-46123",NaN,2022-06-21T00:00:00.000Z
497,"PO-Box-81,-Concord,-NC-28026",NaN,NaN
498,"7400-Windsor-Woods-Dr,-Apt-2C,-Canton,-MI-48187",NaN,NaN


In [224]:
sales_dim['lastSaleDate']

0      2022-03-17T00:00:00.000Z
1      2022-02-08T00:00:00.000Z
2      2018-11-28T00:00:00.000Z
3                           NaN
4                           NaN
                 ...           
495    2016-11-14T00:00:00.000Z
496    2022-06-21T00:00:00.000Z
497                         NaN
498                         NaN
499                         NaN
Name: lastSaleDate, Length: 500, dtype: object

In [236]:
sales_dim['lastSaleDate'] = sales_dim['lastSaleDate'].str.replace('T00:00:00.000Z', '')


In [237]:
sales_dim['lastSaleDate']

0      2022-03-17
1      2022-02-08
2      2018-11-28
3             NaN
4             NaN
          ...    
495    2016-11-14
496    2022-06-21
497           NaN
498           NaN
499           NaN
Name: lastSaleDate, Length: 500, dtype: object

In [238]:
sales_dim.info()
sales_dim.fillna({
    'lastSalePrice' : 0.0,
    'lastSaleDate' : 'unknown'
}, inplace=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             500 non-null    object 
 1   lastSalePrice  211 non-null    float64
 2   lastSaleDate   264 non-null    object 
dtypes: float64(1), object(2)
memory usage: 11.8+ KB


In [281]:
sales_dim.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             500 non-null    object 
 1   lastSalePrice  500 non-null    float64
 2   lastSaleDate   500 non-null    object 
dtypes: float64(1), object(2)
memory usage: 11.8+ KB


## writing my data to csv

In [414]:
location_dim.to_csv('location.csv',mode='w', index=False)
features_dim.to_csv('features.csv', mode='w', index=False)
property_fact.to_csv('property.csv', mode='w', index=False)
Tax_Assessment_dim.to_csv('Tax_Assessment.csv', mode='w',  index=False)
propert_tax_dim.to_csv('property_tax.csv', mode='w', index=False)
owner_dim.to_csv('owner.csv', mode='w', index=False)
sales_dim.to_csv('sales.csv', mode='w', index=False)
print('Data correctly written to csv')

Data correctly written to csv


## Loading my dataset using psycopg2

In [329]:
def get_db_connection():
    connection = psycopg2.connect(
        host='localhost',
        database= 'practice',
        user='postgres',
        password='protocol147'
    )
    return connection

In [ ]:
get_db_connection

In [389]:
def create_tables():
    conn= get_db_connection()
    cursor = conn.cursor()
    create_table_query = '''
                          CREATE SCHEMA IF NOT EXISTS api;

                          DROP TABLE IF EXISTS api.property CASCADE;
                          DROP TABLE IF EXISTS api.Location CASCADE;
                          DROP TABLE IF EXISTS api.Features CASCADE;
                          DROP TABLE IF EXISTS api.Tax_Assessment CASCADE;
                          DROP TABLE IF EXISTS api.property_tax CASCADE;
                          DROP TABLE IF EXISTS api.owner CASCADE;

                          CREATE TABLE IF NOT EXISTS api.Property_fact(
                            id VARCHAR(1000) PRIMARY KEY,
                            assessorID VARCHAR(255),
                            legalDescription VARCHAR(255),
                            ownerOccupied FLOAT,
                            yearBuilt INTEGER,
                            zoning  VARCHAR(255),
                            lotSize FLOAT,
                            propertyType VARCHAR(255),
                            subdivision VARCHAR(255));


                          CREATE TABLE IF NOT EXISTS api.Features(
                            id VARCHAR(1000),
                            architectureType VARCHAR(255),
                            exteriorType VARCHAR(255),
                            floorCount FLOAT,
                            heating VARCHAR(255),
                            viewType VARCHAR(255),
                            cooling VARCHAR(255),
                            coolingType VARCHAR(255),
                            fireplace VARCHAR(255),
                            garage VARCHAR(255),
                            garageType VARCHAR(255),
                            heatingType VARCHAR(255),
                            roofType VARCHAR(255),
                            roomCount FLOAT,
                            unitCount FLOAT,
                            fireplaceType VARCHAR(255),
                            foundationType VARCHAR(255),
                            garageSpaces VARCHAR(255),
                            pool VARCHAR(255),
                            poolType VARCHAR(255),
                            bathrooms FLOAT,
                            bedrooms FLOAT,
                            squareFootage FLOAT,
                            FOREIGN KEY (id) REFERENCES api.Property_fact(id)
                         );

                          CREATE TABLE IF NOT EXISTS api.Tax_Assessment(
                             id VARCHAR(1000),
                             "2021_value" FLOAT,
                             "2021_land" FLOAT,
                             "2021_improvements" FLOAT,
                             "2023_value" FLOAT,
                             "2023_land" FLOAT,
                             "2023_improvements" FLOAT,
                             "2022_value" FLOAT,
                             "2022_land" FLOAT,
                             "2022_improvements" FLOAT,
                             "2020_value" FLOAT,
                             "2020_land" FLOAT,
                             "2020_improvements" FLOAT,
                             "2019_value" FLOAT,
                             "2019_land" FLOAT,
                             "2024_value" FLOAT,
                             "2024_land" FLOAT,
                             "2024_improvements" FLOAT,
                             "2018_value" FLOAT,
                             "2018_land" FLOAT,
                             "2018_improvements" FLOAT,
                             "2019_improvements" FLOAT,
                             "2017_value" FLOAT,
                             "2010_value" FLOAT,
                             "2010_land" FLOAT,
                             "2010_improvements" FLOAT,
                             "2025_value" FLOAT,
                             "2025_land" FLOAT,
                             "2025_improvements" FLOAT,
                             FOREIGN KEY (id) REFERENCES api.Property_fact(id)
                          );


                           CREATE TABLE IF NOT EXISTS api.Property_tax(
                              id VARCHAR(1000),
                              "2020_total" FLOAT,
                              "2021_total" FLOAT,
                              "2022_total" FLOAT,
                              "2023_total" FLOAT,
                              "2019_total" FLOAT,
                              "2017_total" FLOAT,
                              "2018_total" FLOAT,
                              "2016_total" FLOAT,
                              FOREIGN KEY (id) REFERENCES api.Property_fact(id)
                          );


                          CREATE TABLE IF NOT EXISTS api.Owner(
                              id VARCHAR(1000),
                              names VARCHAR(255),
                              mailingAddress_id VARCHAR(1000),
                              mailingAddress_addressLine1 VARCHAR(1000),
                              mailingAddress_city VARCHAR(100),
                              mailingAddress_state VARCHAR(100),
                              mailingAddress_zipCode VARCHAR(100),
                              mailingAddress_formattedAddress VARCHAR(1000),
                              mailingAddress_addressLine2 VARCHAR(1000),
                              FOREIGN KEY (id) REFERENCES api.Property_fact(id)
                          );


                          CREATE TABLE IF NOT EXISTS api.Sales(
                             id VARCHAR(1000), 
                             lastSalePrice FLOAT,
                             lastSaleDate VARCHAR(255),
                             FOREIGN KEY (id) REFERENCES api.Property_fact(id)
                          );


                          CREATE TABLE IF NOT EXISTS api.Location(
                              id VARCHAR(1000),
                              addressLine1 VARCHAR(255),
                              addressLine2 VARCHAR(255),
                              city VARCHAR(255),
                              state VARCHAR(255),
                              zipCode INTEGER,
                              formattedAddress VARCHAR(255),
                              county VARCHAR(255),
                              longitude FLOAT,
                              latitude FLOAT,
                              FOREIGN KEY (id) REFERENCES api.Property_fact(id)
                          );

      
                        '''
    
    cursor.execute(create_table_query)
    conn.commit()
    cursor.close()
    conn.close()                       


In [391]:
create_tables()
print('Schema and Tables created successfully')

Schema and Tables created successfully


In [392]:
## Load csv files to DataBase 
def load_property_fact_csv(file_path):
    conn= get_db_connection()
    cursor = conn.cursor()
    with open(file_path, 'r') as file:
        reader = csv.reader(file)
        next(reader)
        for row in reader:
            cursor.execute(
                '''INSERT INTO api.Property_fact(id,assessorID,legalDescription,ownerOccupied,yearBuilt,zoning,lotSize,propertyType,subdivision)
                    VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s)''', row)
            
            
    conn.commit()
    cursor.close()
    conn.close()

file_path = r'C:\Users\HP\Desktop\Data_Engineer\Api_case_study\property.csv'

load_property_fact_csv(file_path)
print('property_fact data loaded successfully')


property_fact data loaded successfully


In [415]:

def load_features_csv(file_path):
    conn= get_db_connection()
    cursor = conn.cursor()
    with open(file_path, 'r') as file:
        reader = csv.reader(file)
        next(reader)
        for row in reader:
            cursor.execute(
                 '''INSERT INTO api.features(id, architectureType, exteriorType, floorCount, heating, viewType, cooling, coolingType, fireplace,
                                    garage, garageType, heatingType, roofType, roomCount, unitCount,
                                    fireplaceType, foundationType, garageSpaces, pool, poolType,
                                    bathrooms, bedrooms, squareFootage)
                   VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)''', row)
            
            
    conn.commit()
    cursor.close()
    conn.close()

file_path = r'C:\Users\HP\Desktop\Data_Engineer\Api_case_study\features.csv'

load_features_csv(file_path)
print('features data loaded successfully')

features data loaded successfully


In [417]:
def load_location_csv(file_path):
    conn= get_db_connection()
    cursor = conn.cursor()
    with open(file_path, 'r') as file:
        reader = csv.reader(file)
        next(reader)
        for row in reader:
            cursor.execute(
                 '''INSERT INTO api.location(id, addressLine1, addressLine2, city, state, zipCode, formattedAddress, county, longitude,
                                    latitude)
                   VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)''', row)
            
            
    conn.commit()
    cursor.close()
    conn.close()

file_path = r'C:\Users\HP\Desktop\Data_Engineer\Api_case_study\location.csv'

load_location_csv(file_path)
print('location data loaded successfully')

location data loaded successfully


In [421]:
def load_owner_csv(file_path):
    conn= get_db_connection()
    cursor = conn.cursor()
    with open(file_path, 'r') as file:
        reader = csv.reader(file)
        next(reader)
        for row in reader:
            cursor.execute(
                 '''INSERT INTO api.owner(id, names, mailingAddress_id, mailingAddress_addressLine1, mailingAddress_city, mailingAddress_state, mailingAddress_zipCode, 
                                         mailingAddress_formattedAddress, mailingAddress_addressLine2)
                   VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)''', row)
            
            
    conn.commit()
    cursor.close()
    conn.close()

file_path = r'C:\Users\HP\Desktop\Data_Engineer\Api_case_study\owner.csv'

load_owner_csv(file_path)
print("Onwner's data loaded successfully")

Onwner's data loaded successfully


In [424]:
def load_property_tax_csv(file_path):
    conn= get_db_connection()
    cursor = conn.cursor()
    with open(file_path, 'r') as file:
        reader = csv.reader(file)
        next(reader)
        for row in reader:
            cursor.execute(
                 '''INSERT INTO api.Property_tax(id, "2020_total", "2021_total", "2022_total", "2023_total", "2019_total", "2017_total", 
                                         "2018_total", "2016_total")
                   VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)''', row)
            
            
    conn.commit()
    cursor.close()
    conn.close()

file_path = r'C:\Users\HP\Desktop\Data_Engineer\Api_case_study\property_tax.csv'

load_property_tax_csv(file_path)
print("property tax data loaded successfully")

property tax data loaded successfully


In [426]:
def load_sales_csv(file_path):
    conn= get_db_connection()
    cursor = conn.cursor()
    with open(file_path, 'r') as file:
        reader = csv.reader(file)
        next(reader)
        for row in reader:
            cursor.execute(
                 '''INSERT INTO api.Sales(id, lastSalePrice,lastSaleDate )
                   VALUES (%s, %s, %s)''', row)
            
            
    conn.commit()
    cursor.close()
    conn.close()

file_path = r'C:\Users\HP\Desktop\Data_Engineer\Api_case_study\sales.csv'

load_sales_csv(file_path)
print("Sales data loaded successfully")

Sales data loaded successfully


In [428]:
len(Tax_Assessment_dim.columns)

29

In [432]:
def load_Tax_Assessment_csv(file_path):
    conn= get_db_connection()
    cursor = conn.cursor()
    with open(file_path, 'r') as file:
        reader = csv.reader(file)
        next(reader)
        for row in reader:
            cursor.execute(
                 '''INSERT INTO api.Tax_Assessment(id, "2021_value", "2021_land", "2021_improvements", "2023_value", 
                                                 "2023_land", "2023_improvements", "2022_value", "2022_land", 
                                                 "2022_improvements", "2020_value","2020_land","2020_improvements", 
                                                 "2019_value", "2019_land","2024_value" , "2024_land", "2024_improvements",
                                                "2018_value","2018_land", "2018_improvements", "2019_improvements", "2017_value",
                                                "2010_value","2010_land","2010_improvements","2025_value","2025_land","2025_improvements")
                   VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s,%s,%s, %s, %s, %s, %s, %s, %s, %s, %s,%s,%s, %s, %s, %s, %s, %s, %s, %s, %s)''', row)
            
            
    conn.commit()
    cursor.close()
    conn.close()

file_path = r'C:\Users\HP\Desktop\Data_Engineer\Api_case_study\Tax_Assessment.csv'

load_Tax_Assessment_csv(file_path)
print("Tax_Assessment data loaded successfully")

Tax_Assessment data loaded successfully
